In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from pathlib import Path

In [3]:
from surname.dataset import ProjectDataset
from surname.args import args

In [4]:
path = Path('../data/surnames')
scratch = path/'scratch'
surnames_csv = path/args.proc_dataset_csv
vectorizer_path = scratch/args.vectorizer_fname
cw_file = scratch/args.cw_file
args.save_dir = scratch/args.dir_name

args

Namespace(batch_size=64, checkpointer_name='classifier', checkpointer_prefix='surname', cw_file='class_weights.pt', device='cuda:3', dir_name='models', early_stopping_criteria=5, hidden_dim=300, learning_rate=0.001, num_epochs=100, proc_dataset_csv='surnames_with_splits.csv', raw_dataset_csv='surnames.csv', save_dir=PosixPath('../data/surnames/scratch/models'), save_every=2, save_total=5, train_proportion=0.7, vectorizer_fname='vectorizer.json')

In [5]:
df = pd.read_csv(surnames_csv)
df.head()

,nationality,nationality_index,split,surname
0,Arabic,15,train,Totah
1,Arabic,15,train,Abboud
2,Arabic,15,train,Fakhoury
3,Arabic,15,train,Srour
4,Arabic,15,train,Sayegh


In [6]:
is_load = True

In [7]:
if not is_load:
  train_ds = ProjectDataset.load_data_and_create_vectorizer(df.loc[df['split'] == 'train'])
  train_ds.save_vectorizer(vectorizer_path)
  vectorizer = train_ds.get_vectorizer()
  class_counts = df['nationality'].value_counts().to_dict()
  sorted_counts = sorted(class_counts.items(), key=lambda x: vectorizer.nationality_vocab.lookup_token(x[0]))
  freq = [count for _, count in sorted_counts]
  class_weights = 1.0/torch.tensor(freq, dtype=torch.float32)
  torch.save(class_weights, cw_file)

In [8]:
train_df = df.loc[df['split'] == 'train']
train_ds = ProjectDataset.load_data_and_vectorizer(train_df, vectorizer_path)
vectorizer = train_ds.get_vectorizer()
class_weights = torch.load(cw_file)
train_dl = DataLoader(train_ds, args.batch_size, shuffle=True, drop_last=True)

In [9]:
val_df = df.loc[df['split'] == 'val']
val_ds = ProjectDataset.load_data_and_vectorizer(val_df, vectorizer_path)
val_dl = DataLoader(val_ds, args.batch_size, shuffle=True, drop_last=True)

In [10]:
test_df = df.loc[df['split'] == 'test']
test_ds = ProjectDataset.load_data_and_vectorizer(test_df, vectorizer_path)
test_dl = DataLoader(test_ds, args.batch_size, shuffle=True, drop_last=True)

In [11]:
class Classifier(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Classifier, self).__init__()
    self.fc1 = nn.Linear(input_dim, hidden_dim)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_dim, output_dim)
    self.softmax = nn.Softmax()

  def forward(self, x_in, apply_softmax=False):
    y_out = self.fc1(x_in)
    y_out = self.relu(y_out)
    y_out = self.fc2(y_out)
    
    if apply_softmax == True:
      y_out = self.softmax(y_out)

    return y_out

In [12]:
classifier = Classifier(input_dim=len(vectorizer.surname_vocab), hidden_dim=args.hidden_dim, output_dim=len(vectorizer.nationality_vocab))
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
loss_func = nn.CrossEntropyLoss(class_weights)

In [13]:
itr = iter(train_dl)

In [24]:
x,y = next(itr)
y_pred = classifier(x)
loss_func(y_pred, y)

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'target'

In [28]:
y_pred.shape

torch.Size([64, 18])